## Exporting a MobileNetV2 model to CoreML
To use a Fireball model in an iOS application, we can use ```exportToCoreMl``` method. This notebook shows how to use this function to create a CoreML model ready to be deployed in an iOS app. It assumes that a trained MobileNetV2 model already exists in the ```Models``` directory. Please refer to the notebook [Image Classification with MobileNetV2](MobileNetV2.ipynb) for more info about using a pretrained MobileNetV2 model.

Fireball can also export models with reduced number of parameters, pruned models, and quatized models. Please refer to the following notebooks for more information:

- [Reducing number of parameters of MobileNetV2 Model](MobileNetV2-Reduce.ipynb)
- [Pruning MobileNetV2 Model](MobileNetV2-Prune.ipynb)
- [Quantizing MobileNetV2 Model](MobileNetV2-Quantize.ipynb)

Note: Fireball uses the ```coremltools``` python package to export CoreML models. If this is not installed you can just run the following command in a new cell and restart the kernel.
```
%pip install coremltools==3.4
```

## Load a pretrained model

In [1]:
from fireball import Model
from fireball.datasets.imagenet import ImageNetDSet
gpus='0,1,2,3'

# Create the test dataset for evaluation.
testDs = ImageNetDSet.makeDatasets('Test', batchSize=256, preProcessing='Crop256Tf', numWorkers=8)

# orgFileName = "Models/MobileNetV2.fbm"        # Original model
# orgFileName = "Models/MobileNetV2QR.fbm"      # Quantized - Retrained
# orgFileName = "Models/MobileNetV2PR.fbm"      # Pruned - Retrained
# orgFileName = "Models/MobileNetV2PRQR.fbm"    # Pruned - Retrained - Quantized - Retrained
# orgFileName = "Models/MobileNetV2RR.fbm"      # Reduced - Retrained
# orgFileName = "Models/MobileNetV2RRQR.fbm"    # Reduced - Retrained - Quantized - Retrained
# orgFileName = "Models/MobileNetV2RRPR.fbm"    # Reduced - Retrained - Pruned - Retrained
orgFileName = "Models/MobileNetV2RRPRQR.fbm"  # Reduced - Retrained - Pruned - Retrained - Quantized - Retrained

model = Model.makeFromFile(orgFileName, testDs=testDs, gpus=gpus)
model.printLayersInfo()
model.initSession()
results = model.evaluate()


Reading from "Models/MobileNetV2RRPRQR.fbm" ... Done.
Creating the fireball model "MobileNetV2" ... Done.

Scope            InShape       Comments                 OutShape      Activ.   Post Act.        # of Params
---------------  ------------  -----------------------  ------------  -------  ---------------  -----------
IN_IMG                         Image Size: 224x224x3    224 224 3     None                      0          
S1_L1_CONV       224 224 3     KSP: 3 2 0x1x0x1         112 112 32    None                      436        
S1_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L1_DWCN       112 112 32    KSP: 3 1 s               112 112 32    None                      279        
S2_L2_BN         112 112 32                             112 112 32    ReLU     x<6.0            128        
S2_L3_CONV       112 112 32    KSP: 1 1 s               112 112 16    None                      265        
S2_L4_BN         112 112 16 

## Exporting to CoreML
CoreML handles the pre-processing of the images inside the model. The arguments ```rgbBias``` and ```scale``` are used to tell CoreML how to do this pre-processing. The pre-processed image is calculated by CoreML as:

```
processedImage = image * scale + rgbBias
```

For MobileNetV2, the processed images must be in RGB format with values normalized between -1 and 1. So, we are using scale=1/127.5 and rgbBias=-1.0, and setting isBgr to False.

In [2]:
from fireball.datasets.imagenet import ImageNetDSet
import os

cmlFileName = orgFileName.replace('.fbm', '.mlmodel')

model.exportToCoreMl(cmlFileName, classNames=ImageNetDSet.classNames,
                     isBgr=False, scale=1./127.5, rgbBias=-1.0)

orgFileSize = os.stat("Models/MobileNetV2.fbm").st_size
print('Original Model File Size: {:,} bytes'.format(orgFileSize))
fileSize = os.stat(cmlFileName).st_size
print('CoreML Model File Size: {:,} bytes ({:2.2%} of original)'.format(fileSize, fileSize/orgFileSize))


Exporting to CoreML model "Models/MobileNetV2RRPRQR.mlmodel" ... 
    Exported all 27 layers.                               
    Saving to "Models/MobileNetV2RRPRQR.mlmodel" ... Done.
Done (97.75 Sec.)
Original Model File Size: 14,169,739 bytes
CoreML Model File Size: 2,551,140 bytes (18.00% of original)


## Using netron to visualize the exported model
We can now visualize the model's network structure using the "netron" package. If netron is not installed, you can just run the following command in a new cell and restart the kernel.

```
%pip install netron
```

In [3]:
import netron
import platform

if platform.system() == 'Darwin':      # Running on MAC
    netron.start(cmlFileName)   
else:
    import socket
    hostIp = socket.gethostbyname(socket.gethostname())
    netron.start(cmlFileName, address=(hostIp,8084))

Serving 'Models/MobileNetV2RRPRQR.mlmodel' at http://10.21.16.50:8084


## Running inference on the exported model
To verify the exported model, we can now run inference on it. Currently the CoreML runtime is only available on Mac. You also need the pillow package because CoreML only accepts images in this format. If pillow is not installed, you can just run the following command in a new cell and restart the kernel.

```
%pip install pillow
```

In [4]:
assert platform.system() == 'Darwin', "This is only supported when running on Mac!"

import cv2
from PIL import Image
import numpy as np
imageFileName = 'CoffeeMug.jpg'

img = cv2.imread(imageFileName)     # Reads image in BGR order
img = np.float32(img)[..., ::-1]    # Convert to RGB

imgSize = img.shape[:2]
ratio = 256.0/min(imgSize)
newSize = (int(np.round(imgSize[1]*ratio)), int(np.round(imgSize[0]*ratio)))

# Note: INTER_AREA is best when shrinking and CV_INTER_CUBIC is best when enlarging
img = cv2.resize(img, newSize,  interpolation = (cv2.INTER_AREA if ratio<1.0 else cv2.INTER_CUBIC))

# Now crop the center 224x224 image
dw = newSize[0] - 224
dh = newSize[1] - 224
resizedImg = img[dh//2:dh//2+224, dw//2:dw//2+224,:]
pilImage = Image.fromarray( np.uint8(resizedImg) )


import coremltools

model = coremltools.models.MLModel(cmlFileName)
outputDict = model.predict({'InputImage': pilImage}, useCPUOnly=True)

labels, probs = zip(*outputDict['ClassProbs'].items())
top3Idxs = np.argsort(probs)[-3:][::-1]    # Indexes of classes with 3 highest probs (decreasing order)
print('Top-3 Classes (For "%s"):'%(imageFileName))
for i in range(3):
    print('    %s (%f)'%(labels[top3Idxs[i]], probs[top3Idxs[i]])) 

Top-3 Classes (For "CoffeeMug.jpg"):
    coffee_mug (0.918992)
    cup (0.062491)
    pitcher (0.011099)


## Where do I go from here?

[Exporting MobileNetV2 Model to ONNX](MobileNetV2-ONNX.ipynb)

[Exporting MobileNetV2 Model to TensorFlow](MobileNetV2-TF.ipynb)

---

[Fireball Playgrounds](../Contents.ipynb)

[Image Classification with MobileNetV2](MobileNetV2.ipynb)

[Reducing number of parameters of MobileNetV2 Model](MobileNetV2-Reduce.ipynb)

[Pruning MobileNetV2 Model](MobileNetV2-Prune.ipynb)

[Quantizing MobileNetV2 Model](MobileNetV2-Quantize.ipynb)
